<a href="https://colab.research.google.com/github/AdrienHdz/SUMO_Travel_time_estimation/blob/master/xml2csv_Sumo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#!/usr/bin/env python
# Eclipse SUMO, Simulation of Urban MObility; see https://eclipse.org/sumo
# Copyright (C) 2013-2019 German Aerospace Center (DLR) and others.
# This program and the accompanying materials
# are made available under the terms of the Eclipse Public License v2.0
# which accompanies this distribution, and is available at
# http://www.eclipse.org/legal/epl-v20.html
# SPDX-License-Identifier: EPL-2.0

# @file    xml2csv.py
# @author  Adrien Hernandez

import xml.etree.ElementTree as ET
import pandas as pd

# Select Sumo vehroute xml output 
mytree = ET.parse('Quebec_data_2\osm.vehroute.xml')
myroot = mytree.getroot()

print(myroot)

<Element 'routes' at 0x000001F03990D180>


In [11]:
id_trip = []; type_vehicle = []; depart = []; edges = []; exitTimes = []

def extract_data(root):
    for child in root.findall("vehicle"):
        id_trip.append(child.get("id"))
        type_vehicle.append(child.get("type"))
        depart.append(child.get("depart"))
        if child.find("route") is not None:
            rank = child.find('route')
            edges.append(rank.get("edges"))
            exitTimes.append(rank.get("exitTimes"))
        else:
            rank = child.find('routeDistribution')
            rank = rank[-1]
            for rout in rank.iter('route'):
                edges.append(rout.get("edges"))
                exitTimes.append(rout.get("exitTimes"))

In [12]:
extract_data(myroot)

In [13]:
len(id_trip), len(type_vehicle), len(depart), len(edges), len(exitTimes)

(91772, 91772, 91772, 91772, 91772)

In [14]:
id_trip = pd.Series(id_trip); type_vehicle = pd.Series(type_vehicle); depart = pd.Series(depart); edges = pd.Series(edges); exitTimes = pd.Series(exitTimes)

In [15]:
vehroutedata = pd.concat([id_trip, type_vehicle, depart, edges, exitTimes], axis=1)
vehroutedata.columns = ["id", "type", "depart", "edge", "exitTimes"]

In [16]:
print(vehroutedata)

            id  type    depart  \
0         0117  None   1826.00   
1         0136  None   1854.00   
2          014  None   1806.00   
3          016  None   1809.00   
4          010  None   1800.00   
...        ...   ...       ...   
91767  0153732  None  82398.00   
91768  0153694  None  82341.00   
91769  0153683  None  82325.00   
91770  0153702  None  82353.00   
91771  0153668  None  82302.00   

                                                    edge  \
0      -25775336#1 -25775336#0 633452341#0 633452341#...   
1      351535176#1 -351535176#1 -351535176#0 -2517066...   
2      -25482096 25482098 -24411814#12 -24411814#11 -...   
3      -25154426#4 25420525#3 25420525#4 -25482062#3 ...   
4      25420497#2 25420497#3 -25420502 -268183780#1 -...   
...                                                  ...   
91767  17055393#1 377868532#0 377868532#1 164512033#9...   
91768  25465249#13 25465249#14 -25465248#5 -25465248#...   
91769  24411832#5 24411832#6 -264028375 25775363#8 

In [17]:
vehroutedata.to_csv(r'Quebec_data_2\vehroutedata.csv', index = False)